In [4]:

# ------------ PARAMETERS ------------
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

np.random.seed(42)

# ------------ PARAMETERS ------------
N_CUSTOMERS = 500
START_DATE = datetime(2023, 1, 1)
END_DATE = datetime(2024, 12, 31)

# ------------ 1. CUSTOMERS ------------
customer_ids = [f"C{str(i).zfill(4)}" for i in range(1, N_CUSTOMERS + 1)]

industries = ["SaaS", "E-commerce", "Fintech", "Healthtech", "Edtech"]
regions = ["North America", "Europe", "APAC", "LATAM"]
segments = ["SMB", "Mid-Market", "Enterprise"]
acq_channels = ["Paid", "Organic", "Referral", "Partner"]

signup_dates = [
    START_DATE + timedelta(days=int(x)) 
    for x in np.random.uniform(0, (END_DATE - START_DATE).days, N_CUSTOMERS)
]

customers = pd.DataFrame({
    "customer_id": customer_ids,
    "industry": np.random.choice(industries, N_CUSTOMERS),
    "region": np.random.choice(regions, N_CUSTOMERS),
    "segment": np.random.choice(segments, N_CUSTOMERS, p=[0.5, 0.35, 0.15]),
    "signup_date": signup_dates,
    "acquisition_channel": np.random.choice(acq_channels, N_CUSTOMERS)
})


In [8]:
# ------------ 2. SUBSCRIPTIONS ------------
plans = ["Basic", "Pro", "Enterprise"]
plan_type = ["Monthly", "Annual"]

subs_data = []
sub_id_counter = 1

for _, row in customers.iterrows():
    cust_id = row["customer_id"]
    # each customer can have 1–3 subs over time (upgrades/downgrades)
    n_subs = np.random.choice([1, 1, 1, 2, 3])
    sub_start_date = row["signup_date"]
    
    for _ in range(n_subs):
        if sub_start_date > END_DATE:
            break
        
        plan = np.random.choice(plans, p=[0.5, 0.35, 0.15])
        ptype = np.random.choice(plan_type, p=[0.7, 0.3])
        
        if plan == "Basic":
            base_mrr = np.random.randint(50, 150)
        elif plan == "Pro":
            base_mrr = np.random.randint(200, 600)
        else:
            base_mrr = np.random.randint(800, 3000)
        
        # contract length
        if ptype == "Monthly":
            duration_months = np.random.choice([3, 6, 12, 24], p=[0.3, 0.3, 0.3, 0.1])
        else:
            duration_months = np.random.choice([12, 24, 36], p=[0.6, 0.3, 0.1])
        
        end_date = sub_start_date + pd.DateOffset(months=duration_months)
        if end_date > END_DATE:
            end_date = END_DATE
        
        subs_data.append({
            "subscription_id": f"S{str(sub_id_counter).zfill(5)}",
            "customer_id": cust_id,
            "plan_name": plan,
            "plan_type": ptype,
            "contract_mrr": base_mrr,
            "start_date": sub_start_date.date(),
            "end_date": end_date.date(),
            "is_active": int(end_date.date() >= datetime.today().date())

        })
        
        sub_id_counter += 1
        sub_start_date = end_date + timedelta(days=1)
   

subscriptions = pd.DataFrame(subs_data)


In [9]:
# ------------ 3. INVOICES ------------
invoice_rows = []
invoice_id_counter = 1

def month_range(start, end):
    dates = []
    cur = datetime(start.year, start.month, 1)
    while cur <= end:
        dates.append(cur)
        cur = cur + pd.DateOffset(months=1)
    return dates

for _, sub in subscriptions.iterrows():
    sub_id = sub["subscription_id"]
    cust_id = sub["customer_id"]
    start = pd.to_datetime(sub["start_date"])
    end = pd.to_datetime(sub["end_date"])
    mrr = sub["contract_mrr"]
    
    for invoice_month_start in month_range(start, end):
        period_start = invoice_month_start
        period_end = (invoice_month_start + pd.DateOffset(months=1)) - timedelta(days=1)
        if period_start > end:
            break
        
        amount_expected = float(mrr)
        
        # Inject leakage patterns
        # 1) Under-billing
        if np.random.rand() < 0.1:
            amount_billed = amount_expected * np.random.uniform(0.6, 0.95)
        else:
            amount_billed = amount_expected
        
        # 2) Discounts
        base_discount = 0
        if np.random.rand() < 0.3:
            base_discount = amount_billed * np.random.uniform(0.05, 0.25)
        
        # 3) Tax
        tax_amount = amount_billed * 0.1  # 10% tax
        
        gross_amount = amount_billed - base_discount + tax_amount
        
        # 4) Payment behavior
        rnd = np.random.rand()
        if rnd < 0.85:
            payment_status = "Paid"
            amount_paid = gross_amount
        elif rnd < 0.95:
            payment_status = "Partially Paid"
            amount_paid = gross_amount * np.random.uniform(0.3, 0.8)
        else:
            payment_status = "Failed"
            amount_paid = 0.0
        
        invoice_rows.append({
            "invoice_id": f"INV{str(invoice_id_counter).zfill(6)}",
            "customer_id": cust_id,
            "subscription_id": sub_id,
            "invoice_date": period_start.date(),
            "period_start": period_start.date(),
            "period_end": period_end.date(),
            "amount_expected": round(amount_expected, 2),
            "amount_billed": round(amount_billed, 2),
            "discount_applied": round(base_discount, 2),
            "tax_amount": round(tax_amount, 2),
            "amount_paid": round(amount_paid, 2),
            "payment_status": payment_status
        })
        
        invoice_id_counter += 1

invoices = pd.DataFrame(invoice_rows)


In [13]:
# ------------ 4. USAGE EVENTS ------------
usage_rows = []
usage_id_counter = 1

for _, sub in subscriptions.iterrows():
    cust_id = sub["customer_id"]
    sub_id = sub["subscription_id"]
    start = pd.to_datetime(sub["start_date"])
    end = pd.to_datetime(sub["end_date"])
    
    # sample random days of usage
    n_days = np.random.randint(20, 60)
    days = np.random.choice(
        pd.date_range(start, end, freq="D"),
        size=min(n_days, max(1, (end - start).days + 1)),
        replace=False
    )
    
    for day in days:
        # usage pattern varies by segment via customer lookup
        segment = customers.loc[customers["customer_id"] == cust_id, "segment"].values[0]
        if segment == "SMB":
            api_calls = np.random.poisson(50)
        elif segment == "Mid-Market":
            api_calls = np.random.poisson(200)
        else:
            api_calls = np.random.poisson(800)
        
        # Assume 1,000 calls = $10
        billable_units = api_calls
        billable_value = (api_calls / 1000) * 10.0
        
        # Inject unbilled usage: some usage rows will never be fully billed
        if np.random.rand() < 0.15:
            billable_value = billable_value  # potential value
            # actual billing gap will show up when you compare to invoices
        
        usage_rows.append({
    "usage_id": f"U{str(usage_id_counter).zfill(7)}",
    "customer_id": cust_id,
    "subscription_id": sub_id,
    "date": pd.to_datetime(day).date(),  # ← FIX HERE
    "feature_used": "core_api",
    "billable_units": billable_units,
    "billable_value": round(billable_value, 2)
        })
        
        usage_id_counter += 1

usage_events = pd.DataFrame(usage_rows)

In [14]:
# ------------ 5. PAYMENTS ------------
payments_rows = []
payment_id_counter = 1

for _, inv in invoices.iterrows():
    if inv["payment_status"] == "Paid":
        n_payments = 1
    elif inv["payment_status"] == "Partially Paid":
        n_payments = np.random.choice([1, 2])
    else:
        n_payments = 0
    
    remaining = inv["amount_paid"]
    for _ in range(n_payments):
        if remaining <= 0:
            break
        pay = remaining if n_payments == 1 else remaining * np.random.uniform(0.4, 0.8)
        remaining -= pay
        
        payments_rows.append({
            "payment_id": f"P{str(payment_id_counter).zfill(7)}",
            "invoice_id": inv["invoice_id"],
            "payment_date": (pd.to_datetime(inv["invoice_date"]) + timedelta(days=np.random.randint(0, 40))).date(),
            "payment_amount": round(pay, 2),
            "payment_method": np.random.choice(["Card", "ACH", "Wire"]),
            "retry_count": np.random.randint(0, 4)
        })
        payment_id_counter += 1

payments = pd.DataFrame(payments_rows)

# ------------ SAVE TO CSV ------------
customers.to_csv("customers.csv", index=False)
subscriptions.to_csv("subscriptions.csv", index=False)
invoices.to_csv("invoices.csv", index=False)
usage_events.to_csv("usage_events.csv", index=False)
payments.to_csv("payments.csv", index=False)

print("✅ Synthetic data generated and saved as CSVs.")

✅ Synthetic data generated and saved as CSVs.


In [15]:
import pandas as pd
from pathlib import Path

# If everything is in the same folder as the notebook:
data_path = Path(".")

# 1. Load customers
customers = pd.read_csv(
    data_path / "customers.csv",
    parse_dates=["signup_date"]
)

# 2. Load subscriptions
subscriptions = pd.read_csv(
    data_path / "subscriptions.csv",
    parse_dates=["start_date", "end_date"]
)

# 3. Load invoices
invoices = pd.read_csv(
    data_path / "invoices.csv",
    parse_dates=["invoice_date", "period_start", "period_end"]
)

# 4. Load usage events
usage_events = pd.read_csv(
    data_path / "usage_events.csv",
    parse_dates=["date"]
)

# 5. Load payments
payments = pd.read_csv(
    data_path / "payments.csv",
    parse_dates=["payment_date"]
)

customers.head(), subscriptions.head(), invoices.head()


(  customer_id    industry  region     segment signup_date acquisition_channel
 0       C0001        SaaS    APAC  Enterprise  2023-10-01            Referral
 1       C0002        SaaS  Europe         SMB  2024-11-25             Partner
 2       C0003        SaaS    APAC  Enterprise  2024-06-18             Organic
 3       C0004      Edtech  Europe         SMB  2024-03-13            Referral
 4       C0005  Healthtech    APAC  Mid-Market  2023-04-24             Organic,
   subscription_id customer_id plan_name plan_type  contract_mrr start_date  \
 0          S00001       C0001     Basic   Monthly           109 2023-10-01   
 1          S00002       C0001     Basic   Monthly            53 2024-10-02   
 2          S00003       C0002     Basic   Monthly           104 2024-11-25   
 3          S00004       C0003       Pro   Monthly           387 2024-06-18   
 4          S00005       C0004       Pro   Monthly           351 2024-03-13   
 
     end_date  is_active  
 0 2024-10-01         

In [16]:
# Normalize invoice_date to month start, e.g. 2023-05-15 -> 2023-05-01
invoices["month"] = invoices["invoice_date"].dt.to_period("M").dt.to_timestamp()

invoices[["invoice_id", "customer_id", "invoice_date", "month"]].head()


,invoice_id,customer_id,invoice_date,month
0,INV000001,C0001,2023-10-01,2023-10-01
1,INV000002,C0001,2023-11-01,2023-11-01
2,INV000003,C0001,2023-12-01,2023-12-01
3,INV000004,C0001,2024-01-01,2024-01-01
4,INV000005,C0001,2024-02-01,2024-02-01


In [21]:

#We’ll define two leakage types directly from invoices:

#Under-billing: amount_expected - amount_billed if positive

#Unpaid: amount_billed - amount_paid if positive

invoices["underbilling"] = (invoices["amount_expected"] - invoices["amount_billed"]).clip(lower=0)
invoices["unpaid"] = (invoices["amount_billed"] - invoices["amount_paid"]).clip(lower=0)

invoices[[
    "invoice_id", "customer_id", "month",
    "amount_expected", "amount_billed", "amount_paid",
    "underbilling", "unpaid"
]].head()


,invoice_id,customer_id,month,amount_expected,amount_billed,amount_paid,underbilling,unpaid
0,INV000001,C0001,2023-10-01,109.0,109.00,92.77,0.00,16.23
1,INV000002,C0001,2023-11-01,109.0,72.64,70.58,36.36,2.06
2,INV000003,C0001,2023-12-01,109.0,109.00,119.90,0.00,0.00
3,INV000004,C0001,2024-01-01,109.0,109.00,36.56,0.00,72.44
4,INV000005,C0001,2024-02-01,109.0,109.00,0.00,0.00,109.00


In [24]:
#customer monthly leakage
cust_month = (
    invoices
    .groupby(["customer_id", "month"], as_index=False)
    .agg(
        potential_revenue=("amount_expected", "sum"),
        billed_revenue=("amount_billed", "sum"),
        collected_revenue=("amount_paid", "sum"),
        underbilling=("underbilling", "sum"),
        unpaid=("unpaid", "sum")
    )
)

cust_month.head()


,customer_id,month,potential_revenue,billed_revenue,collected_revenue,underbilling,unpaid
0,C0001,2023-10-01,109.0,109.00,92.77,0.00,16.23
1,C0001,2023-11-01,109.0,72.64,70.58,36.36,2.06
2,C0001,2023-12-01,109.0,109.00,119.90,0.00,0.00
3,C0001,2024-01-01,109.0,109.00,36.56,0.00,72.44
4,C0001,2024-02-01,109.0,109.00,0.00,0.00,109.00


In [26]:
# Add total leakage and leakage percentage
cust_month["total_leakage"] = cust_month["underbilling"] + cust_month["unpaid"]

cust_month["leakage_pct_of_potential"] = (
    cust_month["total_leakage"] / cust_month["potential_revenue"]
).replace([pd.NA, float("inf")], 0)  # avoid division by zero issues

cust_month.head()


,customer_id,month,potential_revenue,billed_revenue,collected_revenue,underbilling,unpaid,total_leakage,leakage_pct_of_potential
0,C0001,2023-10-01,109.0,109.00,92.77,0.00,16.23,16.23,0.148899
1,C0001,2023-11-01,109.0,72.64,70.58,36.36,2.06,38.42,0.352477
2,C0001,2023-12-01,109.0,109.00,119.90,0.00,0.00,0.00,0.000000
3,C0001,2024-01-01,109.0,109.00,36.56,0.00,72.44,72.44,0.664587
4,C0001,2024-02-01,109.0,109.00,0.00,0.00,109.00,109.00,1.000000


In [27]:
cust_month = cust_month.merge(
    customers[["customer_id", "segment", "region", "acquisition_channel"]],
    on="customer_id",
    how="left"
)

cust_month.head()


,customer_id,month,potential_revenue,billed_revenue,collected_revenue,underbilling,unpaid,total_leakage,leakage_pct_of_potential,segment,region,acquisition_channel
0,C0001,2023-10-01,109.0,109.00,92.77,0.00,16.23,16.23,0.148899,Enterprise,APAC,Referral
1,C0001,2023-11-01,109.0,72.64,70.58,36.36,2.06,38.42,0.352477,Enterprise,APAC,Referral
2,C0001,2023-12-01,109.0,109.00,119.90,0.00,0.00,0.00,0.000000,Enterprise,APAC,Referral
3,C0001,2024-01-01,109.0,109.00,36.56,0.00,72.44,72.44,0.664587,Enterprise,APAC,Referral
4,C0001,2024-02-01,109.0,109.00,0.00,0.00,109.00,109.00,1.000000,Enterprise,APAC,Referral


In [28]:
# sage-based metrics: 
usage_events["month"] = usage_events["date"].dt.to_period("M").dt.to_timestamp()

usage_month = (
    usage_events
    .groupby(["customer_id", "month"], as_index=False)
    .agg(
        usage_billable_value=("billable_value", "sum"),
        total_api_calls=("billable_units", "sum")
    )
)

usage_month.head()


,customer_id,month,usage_billable_value,total_api_calls
0,C0001,2023-10-01,16.30,1630
1,C0001,2023-11-01,31.42,3142
2,C0001,2023-12-01,23.36,2336
3,C0001,2024-01-01,15.78,1578
4,C0001,2024-02-01,16.65,1665


In [31]:
cust_month.to_csv("customer_monthly_leakage.csv", index=False)
print("✅ Saved customer_monthly_leakage.csv")



✅ Saved customer_monthly_leakage.csv


In [14]:
!pip install pandas langchain openai

In [17]:
#!python revenue_leakage_agent.py


Traceback (most recent call last):
  File "/Users/arjunsingh/Documents/project/revenue_leakage_agent.py", line 310, in <module>
    main()
  File "/Users/arjunsingh/Documents/project/revenue_leakage_agent.py", line 268, in main
    agent = RevenueLeakageAgent(
            ^^^^^^^^^^^^^^^^^^^^
  File "/Users/arjunsingh/Documents/project/revenue_leakage_agent.py", line 108, in __init__
    raise ImportError(
ImportError: LangChain and its dependencies are not installed. Install 'langchain' and 'openai' to use the LLM features.


In [23]:
#import os
#os.environ['OPENAI_API_KEY'] = 'sk-xxxxxxxxxxxxxxxxxx'



In [24]:
#%env OPENAI_API_KEY=sk-xxxxxxxxxxxxxxxxxx


env: OPENAI_API_KEY=sk-xxxxxxxxxxxxxxxxxx


In [29]:
##import langchain
#import openai
#print(langchain.__version__, openai.__version__)


1.1.0 2.8.1
